# Measurement Error Mitigation - Version 2
## Basado en el siguiente tutorial:
https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/noise/3_measurement_error_mitigation.ipynb

In [1]:
##%matplotlib inline
# Import general libraries (needed for functions)
import numpy as np
import time

# Importing standard Qiskit libraries
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer, IBMQ, transpile, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer import noise

# Import measurement calibration functions
from qiskit.utils.mitigation import (complete_meas_cal, tensored_meas_cal,
                                        CompleteMeasFitter, TensoredMeasFitter)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [3]:
# Generate the calibration circuits
qregisters = QuantumRegister(3)
#qubit_list = [2,3,4]
my_calibrations, quantum_states = complete_meas_cal(qr=qregisters, circlabel='mcal')

In [4]:
quantum_states

['000', '001', '010', '011', '100', '101', '110', '111']

In [5]:
# Execute the calibration circuits without noise
backend = Aer.get_backend('qasm_simulator')
Job = execute(my_calibrations, backend=backend, shots=1000)
cal_results = Job.result()

In [6]:
# The calibration matrix without noise is the identity matrix
My_Fitter = CompleteMeasFitter(cal_results, quantum_states, circlabel='mcal')
print(My_Fitter.cal_matrix)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [7]:
# Generate a noise model for the 5 qubits
noise_model = noise.NoiseModel()
for qi in range(5):
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]]) # OOOOOJOOOOOOOOOOOOOO
    noise_model.add_readout_error(read_err, [qi])

In [8]:
# Execute the calibration circuits
backend = Aer.get_backend('qasm_simulator')
JobWithNoise = qiskit.execute(my_calibrations, backend=backend, shots=1000, noise_model=noise_model)
cal_results2 = JobWithNoise.result()

In [9]:
# Calculate the calibration matrix with the noise model
My_Fitter2 = CompleteMeasFitter(cal_results2, quantum_states, circlabel='mcal')
print(My_Fitter2.cal_matrix)

[[0.706 0.196 0.225 0.057 0.2   0.076 0.046 0.018]
 [0.082 0.61  0.021 0.163 0.024 0.164 0.004 0.045]
 [0.09  0.027 0.584 0.176 0.023 0.007 0.157 0.063]
 [0.008 0.072 0.069 0.514 0.001 0.02  0.018 0.147]
 [0.094 0.024 0.018 0.006 0.625 0.148 0.164 0.045]
 [0.008 0.063 0.006 0.018 0.061 0.519 0.019 0.114]
 [0.01  0.001 0.072 0.018 0.055 0.012 0.536 0.156]
 [0.002 0.007 0.005 0.048 0.011 0.054 0.056 0.412]]


In [10]:
# Plot the calibration matrix
#My_Fitter2.plot_calibration()